In [2]:
import json

import os

import pandas as pd
import numpy as np

In [3]:
LANG_CODES_TO_NAMES = {
    "en": "English",
    "ru": "Russian",
    "ko": "Korean",
    "cmn-Hant": "Traditional Chinese",
    "ja": "Japanese",
    "de": "German",
    "es": "Spanish",
}

In [4]:
# Function to get the script directory
def get_script_dir():
    """Returns the directory where the script is located."""
    dir = os.getcwd()
    if dir.endswith("modTiers"):
        return dir[:-len("modTiers")]
    return dir

# Function to generate file path
def get_file_path(base_dir: str, file: str, lang: str, is_en=False):
    return f"{base_dir}{lang if not is_en else 'en'}/{file}.json"

# Function to load a JSON file
def load_file(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    with open(path, encoding="utf-8") as f:
        return json.load(f)

# Function to load a DataFrame
def load_df(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    return pd.read_json(path, encoding="utf-8")

def from_json(base_dir: str, file: str, lang: str, is_en=False):
    file = load_file(base_dir, file, lang, is_en)
    return pd.DataFrame(file)

In [5]:
cwd = get_script_dir()
base_dir = cwd + "/../tables/"
lang = "en"

In [6]:
base_items = from_json(base_dir, "BaseItemTypes", lang).drop(columns=["_index"])
stats = from_json(base_dir, "Stats", lang).drop(columns=["_index"])
runes = from_json(base_dir, "SoulCores", lang).drop(columns=["_index"])

In [7]:
stats

,Id,IsLocal,IsWeaponLocal,Semantic,Text,IsVirtual,HASH32,BelongsActiveSkills,IsScalable
0,level,False,False,3,Level,False,1043633784,[],True
1,item_drop_slots,False,False,3,Item Drop Slots,False,2994048877,[],True
2,main_hand_weapon_type,False,False,3,Main Hand Weapon Type,True,356082681,[],True
3,off_hand_weapon_type,False,False,3,Off Hand Weapon Type,True,501678916,[],True
4,current_endurance_charges,False,False,3,Endurance Charges,False,2172576702,[],True
...,...,...,...,...,...,...,...,...,...
21154,demigods_valor,False,False,4,,False,3121745380,[],True
21155,demigods_fortitude,False,False,4,,False,1289220464,[],True
21156,demigods_judgement,False,False,4,,False,3894467041,[],True
21157,quality_display_banner_buff_effect_+%_final_pe...,False,False,4,,False,1357520477,[],True


In [8]:
def replace_indices_with_ids(index_list, stats_df):
    return [stats_df.loc[i, 'Id'] for i in index_list]

In [9]:
runes['StatsArmour'] = runes['StatsArmour'].apply(lambda lst: replace_indices_with_ids(lst, stats))
runes['StatsWeapon'] = runes['StatsWeapon'].apply(lambda lst: replace_indices_with_ids(lst, stats))
runes

,BaseItemType,StatsWeapon,StatsValuesWeapon,StatsArmour,StatsValuesArmour
0,491,"[local_minimum_added_fire_damage, local_maximu...","[7, 11]",[base_fire_damage_resistance_%],[12]
1,492,"[local_minimum_added_cold_damage, local_maximu...","[6, 10]",[base_cold_damage_resistance_%],[12]
2,493,"[local_minimum_added_lightning_damage, local_m...","[1, 20]",[base_lightning_damage_resistance_%],[12]
3,494,[local_physical_damage_+%],[20],[local_armour_and_evasion_and_energy_shield_+%],[20]
4,495,[local_life_leech_from_physical_damage_permyriad],[300],[base_maximum_life],[25]
5,496,[local_mana_leech_from_physical_damage_permyriad],[200],[base_maximum_mana],[20]
6,497,[base_life_gained_on_enemy_death],[20],[life_regeneration_rate_per_minute_%],[18]
7,498,[base_mana_gained_on_enemy_death],[10],[mana_regeneration_rate_+%],[15]
8,499,[local_hit_damage_stun_multiplier_+%],[25],[stun_threshold_+],[40]
9,500,[local_accuracy_rating],[100],[flask_life_and_mana_to_recover_+%],[10]


In [10]:
runes_merged = runes.merge(base_items, left_on="BaseItemType", right_index=True, how="left")
runes_merged[["BaseItemType", "StatsWeapon", "StatsValuesWeapon", "StatsArmour", "StatsValuesArmour"]]

,BaseItemType,StatsWeapon,StatsValuesWeapon,StatsArmour,StatsValuesArmour
0,491,"[local_minimum_added_fire_damage, local_maximu...","[7, 11]",[base_fire_damage_resistance_%],[12]
1,492,"[local_minimum_added_cold_damage, local_maximu...","[6, 10]",[base_cold_damage_resistance_%],[12]
2,493,"[local_minimum_added_lightning_damage, local_m...","[1, 20]",[base_lightning_damage_resistance_%],[12]
3,494,[local_physical_damage_+%],[20],[local_armour_and_evasion_and_energy_shield_+%],[20]
4,495,[local_life_leech_from_physical_damage_permyriad],[300],[base_maximum_life],[25]
5,496,[local_mana_leech_from_physical_damage_permyriad],[200],[base_maximum_mana],[20]
6,497,[base_life_gained_on_enemy_death],[20],[life_regeneration_rate_per_minute_%],[18]
7,498,[base_mana_gained_on_enemy_death],[10],[mana_regeneration_rate_+%],[15]
8,499,[local_hit_damage_stun_multiplier_+%],[25],[stun_threshold_+],[40]
9,500,[local_accuracy_rating],[100],[flask_life_and_mana_to_recover_+%],[10]


In [11]:
row = runes_merged[runes_merged["Name"] == "Iron Rune"][
        ["StatsWeapon", "StatsValuesWeapon", "StatsArmour", "StatsValuesArmour"]
    ]
row.to_dict('records')[0]

{'StatsWeapon': ['local_physical_damage_+%'],
 'StatsValuesWeapon': [20],
 'StatsArmour': ['local_armour_and_evasion_and_energy_shield_+%'],
 'StatsValuesArmour': [20]}